In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
classes = ['dry_asphalt_severe',
           'dry_asphalt_smooth',
           'dry_concrete_severe',
           'dry_concrete_smooth',
           'dry_gravel',
           'wet_asphalt_severe',
           'wet_asphalt_smooth']

batch_size = 64

num_classes =len(classes)

dataset_path = '/kaggle/input/rts-7-classes'

In [3]:
from keras.preprocessing.image import ImageDataGenerator

def generators(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest',
        validation_split = 0.2,
    )

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        dataset_path,
        target_size = (height, width), 
        classes = classes,
        batch_size = batch_size,
        subset = 'training', 
    )

    val_dataset = imgdatagen.flow_from_directory(
        dataset_path,
        target_size = (height, width), 
        classes = classes,
        batch_size = batch_size,
        subset = 'validation'
    )
    return train_dataset, val_dataset

In [6]:
train_dataset, val_dataset = generators(
    (224, 224), preprocessing=keras.applications.resnet50.preprocess_input)

Found 47017 images belonging to 7 classes.
Found 11754 images belonging to 7 classes.


In [15]:
inputs = keras.Input(shape=(224, 224, 3))

"""Loading the ResNet50 model with pre-trained ImageNet weights
"""
resnet = keras.applications.ResNet50(weights='imagenet',pooling='avg', classes=num_classes,
                                 include_top=False, input_tensor=inputs)

for layer in resnet.layers:
    layer.trainable = False

In [17]:
model = keras.models.Sequential()
model.add(resnet)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(num_classes, activation='softmax'))

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 7)                 3591      
                                                                 
Total params: 24640391 (94.00 MB)
Trainable params: 1052679 (4.02 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
checkpointer = keras.callbacks.ModelCheckpoint(filepath='best_weight.h5',
                                           monitor="val_accuracy",
                                           verbose=1, save_best_only=True)

In [27]:
history = model.fit(train_dataset, validation_data=val_dataset,
                    epochs=15, callbacks=[checkpointer], workers=4)

Epoch 1/15
735/735 [==============================] - ETA: 0s - loss: 0.6691 - accuracy: 0.7522
Epoch 1: val_accuracy improved from -inf to 0.54552, saving model to best_weight.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


735/735 [==============================] - 443s 594ms/step - loss: 0.6691 - accuracy: 0.7522 - val_loss: 1.3824 - val_accuracy: 0.5455
Epoch 2/15
735/735 [==============================] - ETA: 0s - loss: 0.5141 - accuracy: 0.8064
Epoch 2: val_accuracy improved from 0.54552 to 0.57640, saving model to best_weight.h5
735/735 [==============================] - 470s 636ms/step - loss: 0.5141 - accuracy: 0.8064 - val_loss: 1.2832 - val_accuracy: 0.5764
Epoch 3/15
735/735 [==============================] - ETA: 0s - loss: 0.4627 - accuracy: 0.8236
Epoch 3: val_accuracy improved from 0.57640 to 0.57648, saving model to best_weight.h5
735/735 [==============================] - 411s 557ms/step - loss: 0.4627 - accuracy: 0.8236 - val_loss: 1.3751 - val_accuracy: 0.5765
Epoch 4/15
735/735 [==============================] - ETA: 0s - loss: 0.4311 - accuracy: 0.8378
Epoch 4: val_accuracy did not improve from 0.57648
735/735 [==============================] - 408s 553ms/step - loss: 0.4311 - accura

In [28]:
model.save("rts.h5")

In [40]:
# Visualization
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.manifold import TSNE
import numpy as np
# Visualize Training and Validation Results

# Create Subplot
fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=["Model Loss", "Model Accuracy", "Model Validation Accuracy"], 
)

# Configuration Plot
class PlotCFG:
    marker_size = 8
    line_size = 2
    train_color = "#76503d"
    valid_color = "#deb392"

# Loss Plot
loss = history.history['loss']
val_loss = history.history['val_loss']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(loss)+1), y=loss,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.train_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.train_color, width=PlotCFG.line_size),
        name="Training Loss"
    ), row=1, col=1
)


# Accuracy Plot
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(acc)+1), y=acc,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.train_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.train_color, width=PlotCFG.line_size),
        name="Training Accuracy"
    ), row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(val_acc)+1), y=val_acc,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.train_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.train_color, width=PlotCFG.line_size),
        name="Training Validation Accuracy"
    ), row=1, col=3
)


# Update Axes
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=2)
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=3)
fig.update_yaxes(title="Categorical Loss", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_yaxes(title="Accuracy", linecolor="Black", ticks="outside", row=1, col=2)
fig.update_yaxes(title="Validation Accuracy", linecolor="Black", ticks="outside", row=1, col=3)

# Update Layout
fig.update_layout(
    title="Training Loss and Metrics", title_x=0.5,
    width=1400, height=400,
    showlegend=False,
    plot_bgcolor="White",
    paper_bgcolor="White"
)

# Show
fig.show(iframe_connected=True)

In [31]:
"""
{'loss': [0.6690629124641418,
  0.5141015648841858,
  0.46271705627441406,
  0.4310910105705261,
  0.4137895703315735,
  0.3932514190673828,
  0.38049158453941345,
  0.36860325932502747,
  0.35877981781959534,
  0.34868043661117554,
  0.34983915090560913,
  0.33298805356025696,
  0.3269141614437103,
  0.3268711268901825,
  0.315024197101593],
 'accuracy': [0.7521534562110901,
  0.8064104318618774,
  0.823574423789978,
  0.8378033638000488,
  0.8433758020401001,
  0.8511176705360413,
  0.8569878935813904,
  0.8603058457374573,
  0.8643256425857544,
  0.8687921166419983,
  0.8698555827140808,
  0.8738541603088379,
  0.877448558807373,
  0.8781930208206177,
  0.8818512558937073],
 'val_loss': [1.3824472427368164,
  1.2831628322601318,
  1.3750760555267334,
  1.5347179174423218,
  1.451094388961792,
  1.6488137245178223,
  1.4453610181808472,
  1.6968278884887695,
  1.595888376235962,
  1.8041080236434937,
  1.7349298000335693,
  1.625914216041565,
  1.8319491147994995,
  1.6154347658157349,
  1.953277349472046],
 'val_accuracy': [0.5455164313316345,
  0.5763995051383972,
  0.5764846205711365,
  0.5709545612335205,
  0.6140037178993225,
  0.586523711681366,
  0.6003913283348083,
  0.5936702489852905,
  0.6037944555282593,
  0.5772503018379211,
  0.6037093997001648,
  0.6134932637214661,
  0.601837694644928,
  0.6208950281143188,
  0.6148545145988464]}
"""
history.history


{'loss': [0.6690629124641418,
  0.5141015648841858,
  0.46271705627441406,
  0.4310910105705261,
  0.4137895703315735,
  0.3932514190673828,
  0.38049158453941345,
  0.36860325932502747,
  0.35877981781959534,
  0.34868043661117554,
  0.34983915090560913,
  0.33298805356025696,
  0.3269141614437103,
  0.3268711268901825,
  0.315024197101593],
 'accuracy': [0.7521534562110901,
  0.8064104318618774,
  0.823574423789978,
  0.8378033638000488,
  0.8433758020401001,
  0.8511176705360413,
  0.8569878935813904,
  0.8603058457374573,
  0.8643256425857544,
  0.8687921166419983,
  0.8698555827140808,
  0.8738541603088379,
  0.877448558807373,
  0.8781930208206177,
  0.8818512558937073],
 'val_loss': [1.3824472427368164,
  1.2831628322601318,
  1.3750760555267334,
  1.5347179174423218,
  1.451094388961792,
  1.6488137245178223,
  1.4453610181808472,
  1.6968278884887695,
  1.595888376235962,
  1.8041080236434937,
  1.7349298000335693,
  1.625914216041565,
  1.8319491147994995,
  1.615434765815734

In [47]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict(val_dataset)
y_pred = np.argmax(Y_pred, axis=1)

184/184 [==============================] - 188s 1s/step


In [48]:
print('Confusion Matrix')
print(confusion_matrix(val_dataset.classes, y_pred))
print('Classification Report')
print(classification_report(val_dataset.classes, y_pred, target_names=classes))

Confusion Matrix
[[278 180  89 197 220  54 304]
 [415 235 161 317 356  75 441]
 [450 225 151 311 353  91 419]
 [449 218 151 328 326  72 456]
 [441 218 142 333 334  74 458]
 [ 93  51  29  75  73  15  96]
 [436 219 151 339 373  68 414]]
Classification Report
                     precision    recall  f1-score   support

 dry_asphalt_severe       0.11      0.21      0.14      1322
 dry_asphalt_smooth       0.17      0.12      0.14      2000
dry_concrete_severe       0.17      0.08      0.11      2000
dry_concrete_smooth       0.17      0.16      0.17      2000
         dry_gravel       0.16      0.17      0.17      2000
 wet_asphalt_severe       0.03      0.03      0.03       432
 wet_asphalt_smooth       0.16      0.21      0.18      2000

           accuracy                           0.15     11754
          macro avg       0.14      0.14      0.13     11754
       weighted avg       0.16      0.15      0.15     11754

